In [1]:
# Step 1. Import the required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

In [2]:
# Step 2. Download and initial review of data
train_data = pd.read_csv("final_proj_data.csv", index_col=0)
train_data.head()

test_data = pd.read_csv("final_proj_test.csv", index_col=0)
test_data.head()

,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,Var41,...,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
Var1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,200.0,NaN,2.0,56.0,NaN,NaN,403.36,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4800036.0,NaN,NaN,NaN,...,NaN,75lTmBtFkL,rEUOq2QD1qfkRr6qpua,NaN,taul,1K8T,F9g8,fhk21Ss,GVWDufEPyV,3VuKz0_,NaN,GHne,9_Y1,c2JP,VpdQ,hAFG,7M47J5GA0pTYIFxg5uy,kIsH,NaN,uKAI,L84s,oOT00mkwDoEFW,NaN,XZ6TpvF,NaN,pMW8rnA,6ITF,cJvF,FzaX,4UxGlow,zCkv,catzS2D,LM8l689qOp,NaN,ELof,rgKb,ZI9m,ib5G6X1eUxUn6,am7c,NaN
NaN,NaN,NaN,NaN,NaN,3192.0,28.0,NaN,NaN,NaN,NaN,NaN,3864.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380.0,475.0,NaN,8.0,288.0,NaN,NaN,200.00,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,51282.0,NaN,NaN,NaN,...,NaN,1YVgUHXZeb,RO12,NaN,taul,1K8T,0Xwj,Ml6kiFp,S85OeCBjSz,NaN,NaN,14hG,HLqf,8BBI,09_Q,43pnToF,me75fM6ugJ,sBgB,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,mAjbk_S,RRN_,UYBR,FzaX,k5O88FJ,oslk,QkgQQMs,LM8l689qOp,NaN,NaN,453m,RAYp,F2FyR07IdsN7I,am7c,NaN
NaN,NaN,NaN,NaN,NaN,756.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.0,195.0,NaN,2.0,40.0,NaN,NaN,402.24,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,3462336.0,NaN,NaN,NaN,...,NaN,cFHt0GoOh3,RO12,NaN,taul,1K8T,wGW5,Wq69B7S,z7GH4Up,NaN,NaN,dK_l,9_Y1,rGJy,VpdQ,zm5i,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,NaN,kZJyVg2,c4pP,UYBR,FzaX,h72uX8Z,oslk,bxCQb98,jySVZNlOJy,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN
NaN,NaN,NaN,NaN,NaN,3892.0,21.0,NaN,NaN,NaN,NaN,NaN,5192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.0,190.0,NaN,0.0,8.0,NaN,NaN,212.08,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,7331880.0,NaN,NaN,NaN,...,NaN,m9SrEy7Rm6,RO12,NaN,taul,1K8T,JLbT,4JEMGuV,oQ173939hSRjL,YtLixTf,NaN,zqqA,9_Y1,X8zP,VpdQ,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,xdDXvoB,NaN,mAjbk_S,fgNW,UYBR,FzaX,VA_88FJ,oslk,0JeRt72,M_8D,NaN,ELof,Aoh3,RAYp,F2FyR07IdsN7I,mj86,NaN
NaN,NaN,NaN,NaN,NaN,672.0,7.0,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,180.0,NaN,2.0,64.0,NaN,NaN,322.64,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,381480.0,NaN,NaN,NaN,...,NaN,BQavyxLp3Z,RO12,NaN,taul,1K8T,lK27,32pCIek,JQFeOYK,FWqY20R,NaN,YqMO,9_Y1,_r21,VpdQ,kxE9,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,jrs8zOR,NaN,XTbPUYD,dvCm,UYBR,FzaX,FB4YlW4,oslk,0p8OTZB,LM8l689qOp,NaN,ELof,xb3V,RAYp,F2FyR07IdsN7I,mj86,NaN


In [3]:
# Step 3. Checking data types and missing values
print("Train data types:\n", train_data.dtypes)
print("Test data types:\n", test_data.dtypes)

print("Train data missing values:\n", train_data.isnull().sum())
print("Test data missing values:\n", test_data.isnull().sum())

Train data types:
 Var2      float64
Var3      float64
Var4      float64
Var5      float64
Var6      float64
           ...   
Var227     object
Var228     object
Var229     object
Var230    float64
y           int64
Length: 230, dtype: object
Test data types:
 Var2      float64
Var3      float64
Var4      float64
Var5      float64
Var6      float64
           ...   
Var226     object
Var227     object
Var228     object
Var229     object
Var230    float64
Length: 229, dtype: object
Train data missing values:
 Var2       9734
Var3       9734
Var4       9720
Var5       9759
Var6       1020
          ...  
Var227        0
Var228        0
Var229     5561
Var230    10000
y             0
Length: 230, dtype: int64
Test data missing values:
 Var2      2422
Var3      2422
Var4      2428
Var5      2439
Var6       271
          ... 
Var226       0
Var227       0
Var228       0
Var229    1423
Var230    2500
Length: 229, dtype: int64


In [4]:
# Step 4. Remove unnecessary columns and divide the data into features (X) and labels (y)
features_to_drop = ['Var8', 'Var15', 'Var20', 'Var31', 'Var32', 'Var39', 'Var42', 'Var48', 'Var52',
                    'Var55', 'Var79', 'Var141', 'Var167', 'Var169', 'Var175', 'Var185', 'Var209', 'Var230']
train_data.drop(columns=features_to_drop, inplace=True)
test_data.drop(columns=features_to_drop, inplace=True)

X = train_data.drop(columns=['y'])
y = train_data['y']

In [5]:
# Step 5. Using SimpleImputer, StandardScaler, and OneHotEncoder to process numeric and categorical data
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [6]:
# Step 6. Combining numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [7]:
# Step 7. Building and training the model
model = RandomForestClassifier(random_state=42)

pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', model)])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_val_pred = pipeline.predict(X_val)

balanced_acc = balanced_accuracy_score(y_val, y_val_pred)
print(f'Balanced Accuracy: {balanced_acc:.2f}')

Balanced Accuracy: 0.68


In [8]:
# Step 8. Prediction and file creation

y_test_pred = pipeline.predict(test_data)

submission = pd.DataFrame({'y': y_test_pred})
submission.to_csv('submission.csv', index_label="index")

# Conclusions

***A value of 0.68 means that the model correctly classifies about 68% of the positive and negative classes in the validation dataset.***

## Strategies used to fill in missing values:

*There are several methods to recover data: mean, median, most_frequent, constant.*

- The median (‘median’) is the average value in an ordered data set that is not affected by extreme values, as can be the case with the mean, so it is the best choice for numeric features in this model.
- The use of a constant (‘constant’) to fill in missing values in categorical features allowed us to retain the information that the value was missing. This method allows you to fill in the missing values with a specific constant, such as ‘missing’, which makes it possible to clearly mark the missing values as a separate category. This can be useful for the model, as the missing data may have a specific meaning.

Also, when one of the categories (e.g., positive or negative class) in the dataset is significantly larger than the other, the model may refocus on the more frequent class, ignoring the less represented one. This leads to poor model performance on the less represented class, even if the overall accuracy looks high. Therefore, to eliminate this problem, SMOTE was used to create new data located on the segments between the points of the less represented class. This allowed us to ‘balance’ the distribution of classes and train the model to better recognise all categories.

## Results

*Model predictions are stored in the **submission.csv** file, which contains the predicted churn status for each client in the test set.*